In [1]:
import librosa, librosa.display 
import numpy as np
import glob
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch
import matplotlib.pyplot as plt
from sklearn.preprocessing import MaxAbsScaler

In [2]:
import glob
import cv2 as cv
import pandas as pd
from PIL import Image
images = glob.glob('/home/vislab/Codes/DoLearn_jimin/data/image_plt/*.png')
label=pd.read_csv('/home/vislab/Codes/DoLearn_jimin/data/y_label.csv')
print(len(images))

332


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [4]:
mel_images=[]
mel_labels=[]
i=0
for fname in images:
    image_pil = Image.open(fname).convert('RGB')
    image_bw = np.array(image_pil)
    mel_images.append(image_bw)
    i+=1

mel_images=np.array(mel_images)

In [5]:
#mel_images=np.expand_dims(mel_images, 1)

In [6]:
mel_images=np.transpose(mel_images,(0,3,1,2))

In [7]:
print(mel_images.shape)

(332, 3, 128, 1876)


In [8]:
#plt.imshow(mel_images[0])

In [9]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

def normalize2(image):
    image = (image - mean) / std
    # in addition, roll the axis so that they suit pytorch
    return torch.tensor(image.swapaxes(-1, 1).swapaxes(2, 3)).float().to(device)

In [10]:
# load the model
from torchvision import models
model = models.vgg16(pretrained=True).eval()
model.to(device)

model.classifier[6].out_features=2
model.features[0].in_channels=1

print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [11]:
x_train, x_test, y_train, y_test = train_test_split(mel_images, label['0'], test_size=0.33, random_state=42, stratify = label['0'])

In [12]:
import random

random.seed(1)

x_train=torch.FloatTensor(np.array(x_train)).cuda()
y_train=torch.LongTensor(np.array(y_train)).cuda()
x_test=torch.FloatTensor(np.array(x_test)).cuda()
y_test=torch.LongTensor(np.array(y_test)).cuda()

In [13]:
criterion = torch.nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [18]:
train_dataset=torch.utils.data.TensorDataset(x_train,y_train)
train_loader=torch.utils.data.DataLoader(train_dataset,batch_size=128)

In [19]:
from sklearn.metrics import accuracy_score
model.train()
for i in range(15):
    for X, Y in train_loader:
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
    print(i,cost.item(),accuracy_score(Y.cpu(),hypothesis.cpu().detach().numpy().argmax(1)))

RuntimeError: CUDA out of memory. Tried to allocate 612.00 MiB (GPU 0; 23.70 GiB total capacity; 21.39 GiB already allocated; 72.62 MiB free; 21.63 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
x_train.shape

In [ ]:
import json
import glob

label_list = []
i = 1
label = glob.glob('C:\Users\vislab\Documents\두런두런\permission to safe\label\*.json',recursive=True)
for label_path in label:
    with open(label_path,'r',encoding="UTF-8") as f:
        json_data = json.load(f)
    #print(json.dumps(json_data,indent = "\t",ensure_ascii=False))

    One = json_data['annotations'][0]['categories']['category_02']
    label_list.append(One)
    i+=1
    if (i%100 == 0):
        print(i)


In [ ]:
len(label_list)

In [ ]:
y_data = np.array(label_list)

In [ ]:
y_data[y_data=='실외'] = 1
y_data[y_data=='실내'] = 1
y_data[y_data!='1'] = 0
y_data = y_data.astype(int)
print(y_data)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_train, y_data, test_size=0.33, random_state=42, stratify = y_data)

In [ ]:
import random

random.seed(1)

x_train = x_train.astype('float')
x_test = x_test.astype('float')
y_train = y_train.astype('int')
y_test = y_test.astype('int')


In [ ]:
print(x_train.shape)
print(x_test.shape)

In [ ]:
import torch
import torch.nn as nn

seed = 1
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

class ParallelModel(nn.Module):
    def __init__(self,num_emotions):
        super().__init__()

            # 1. conv block
        self.relu = nn.ReLU()
        self.conv1= nn.Conv2d(in_channels=1,
                   out_channels=16,
                   kernel_size=3,
                   stride=1,
                   padding=1
                  )
        self.bn1 = nn.BatchNorm2d(16)

        self.mp1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.do = nn.Dropout2d(p=0.3)
        # 2. conv block
        self.conv2= nn.Conv2d(in_channels=16,
                   out_channels=32,
                   kernel_size=3,
                   stride=1,
                   padding=1
                  )
        self.bn2 = nn.BatchNorm2d(32)
        self.mp2 = nn.MaxPool2d(kernel_size=4, stride=4)

        # 3. conv block
        self.conv3 = nn.Conv2d(in_channels=32,
                   out_channels=64,
                   kernel_size=3,
                   stride=1,
                   padding=1
                  )
        self.bn3 = nn.BatchNorm2d(64)

        # 4. conv block
        self.conv4= nn.Conv2d(in_channels=64,
                   out_channels=128,
                   kernel_size=3,
                   stride=1,
                   padding=1
                  )
        self.bn4= nn.BatchNorm2d(128)

        # Linear softmax layer
        self.out_linear = nn.Linear(1792,num_emotions)
        self.out_softmax = nn.Softmax(dim=1)
        
    def forward(self,x):

        # transformer embedding
        out = self.relu(self.bn1(self.conv1(x)))

        out = self.do(self.mp1(out))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.do(self.mp2(out))
        out = self.relu(self.bn3(self.conv3(out)))
        out = self.do(self.mp2(out))
        out = self.relu(self.bn4(self.conv4(out)))
        out = self.do(self.mp2(out))


        conv_embedding = torch.flatten(out, start_dim=1)

        output_logits = self.out_linear(conv_embedding)
        output_softmax = self.out_softmax(output_logits)
        return output_logits, output_softmax  
    
model = ParallelModel(num_emotions=2).to(device)
print('Number of trainable params: ',sum(p.numel() for p in model.parameters()) )


In [ ]:
EPOCHS=25
DATASET_SIZE = x_train.shape[0]
BATCH_SIZE = 64

##baseline에서 사용한 optimizer는 SGD이며 하이퍼파라미터는 다음과 같습니다.
##Learning rate = 0.01, momentum=0.9
OPTIMIZER = torch.optim.SGD(model.parameters(),lr=0.01, momentum=0.9)

In [ ]:
def loss_fnc(predictions, targets):
    return nn.CrossEntropyLoss()(input=predictions,target=targets)

In [ ]:
!pip install livelossplot

In [ ]:
from livelossplot import PlotLosses
liveloss = PlotLosses()

logs = {}
train_predicts = []

model.train()

for epoch in range(EPOCHS):
    # shuffle data
    ind = np.random.permutation(DATASET_SIZE)

    x_train = x_train[ind,:,:,:]
    y_train = y_train[ind]

    
    #print(x_train.shape)
    #print(y_train.shape)
    
    epoch_loss = 0
    
    iters = int(DATASET_SIZE / BATCH_SIZE)
     
    for i in range(iters):
        ### indexing과정을 통해 작성된 Dataloader 코드
        batch_start = i * BATCH_SIZE
        batch_end = min(batch_start + BATCH_SIZE, DATASET_SIZE)
        actual_batch_size = batch_end-batch_start
        
        x = x_train[batch_start:batch_end,:,:,:]
        y = y_train[batch_start:batch_end]

        #print(x.shape)
        #print(y.shape)
        
        ###--------------------작성해야할 부분--------------------------------
        ##1.train 데이터를 torch tensor로 타입 변경 및 gpu 설정
        ##2.데이터를 모델의 입력으로 넣어 예측 후 loss 계산 및 optimization 
        x_tensor = torch.tensor(x,device=device).float()
        y_tensor = torch.tensor(y, dtype=torch.long, device=device)

        #print(x_tensor.shape)
        #print(y_tensor.shape)
        
        # forward pass
        output_logits, output_softmax = model(x_tensor)

        train_predictions = torch.argmax(output_softmax,dim=1).cpu().numpy()
        train_predicts.append(train_predictions)

        # compute loss
        loss = loss_fnc(output_logits, y_tensor)
        
        # compute gradients
        OPTIMIZER.zero_grad()
        loss.backward()
        # update parameters
        OPTIMIZER.step()

        epoch_loss += loss.item()*actual_batch_size/DATASET_SIZE

    ###liveloss 함수를 사용하기 위한 코드들
    logs['train_loss'] = epoch_loss
        
    liveloss.update(logs)
    liveloss.draw()

    print(epoch,loss.item(),accuracy_score(y, train_predictions))

In [ ]:
from tqdm import tqdm

model.eval()

predicts = []

with torch.no_grad():
    for data in tqdm(x_test):
        data = torch.FloatTensor(data).to(device).unsqueeze(1)

        _ , output_softmax = model(data)
        predictions = torch.argmax(output_softmax,dim=1).cpu().numpy()
        predicts.append(predictions)

print(f'predict_len:{len(predicts)}')

In [ ]:
#model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

#model.fit(x_train, y_train, batch_size=100, epochs=2)

from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, predicts))



In [ ]:
!pip install lime

In [ ]:
from lime import lime_image # 이미지 모듈
from lime.wrappers.scikit_image import SegmentationAlgorithm # LIME이 사용할 수 있게. 

explainer=lime_image.LimeImageExplainer()

In [ ]:
segmenter=SegmentationAlgorithm('slic', 
                               n_segments=100, #이미지 분할 조각 개수
                               compactness=1, # 유사한 파트를 합치는 함수, log scale, default : 10
                               sigma=1) # 스무딩 역할, 0과 1 사이의 float

In [ ]:
mel_spec_db.shape

In [ ]:
img.shape

In [ ]:
olivetti_test_index=0

exp=explainer.explain_instance(img, #데이터하나
                              classifier_fn=model,# 2 class의 확률 반환 (softmax)
                              top_labels=2, # 확률 기준 1-5위 분석
                              num_samples=1000,# 샘플링 공간
                              segmentation_fn=segmenter) # 이전에 lime의 형식에 맞게 래핑된 분할 알고리즘

In [ ]:
x_test[0].shape